In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 13.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from datasets import load_dataset
dataset = load_dataset("cfilt/iitb-english-hindi")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
source_train_file = open("source_train.txt", "w+", encoding='utf8')
target_train_file = open("target_train.txt", "w+", encoding='utf8')
for translation_pair in dataset["train"]["translation"]:
  source_sentence = translation_pair["en"]
  target_sentence = translation_pair["hi"]
  source_train_file.write(source_sentence.strip("\n") + "\n")
  target_train_file.write(target_sentence.strip("\n") + "\n")
source_train_file.close()
target_train_file.close()

source_valid_file = open("source_valid.txt", "w+", encoding='utf8')
target_valid_file = open("target_valid.txt", "w+", encoding='utf8')
for translation_pair in dataset["validation"]["translation"]:
  source_sentence = translation_pair["en"]
  target_sentence = translation_pair["hi"]
  source_valid_file.write(source_sentence.strip("\n") + "\n")
  target_valid_file.write(target_sentence.strip("\n") + "\n")
source_valid_file.close()
target_valid_file.close()

source_test_file = open("source_test.txt", "w+", encoding='utf8')
target_test_file = open("target_test.txt", "w+", encoding='utf8')
for translation_pair in dataset["test"]["translation"]:
  source_sentence = translation_pair["en"]
  target_sentence = translation_pair["hi"]
  source_test_file.write(source_sentence.strip("\n") + "\n")
  target_test_file.write(target_sentence.strip("\n") + "\n")
source_test_file.close()
target_test_file.close()

In [ ]:
hi = []
for i in dataset['train']['translation']:
  hindi = dict()
  hindi['text'] = i['hi']
  hi.append(hindi)
hi

In [ ]:
hi = iter(hi)
hi

In [ ]:
count = 0
for row in hi:
    print(row)
    count += 1
    if count == 3: break

{'text': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}
{'text': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'}
{'text': 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका'}


In [ ]:
def hi_text():
    for row in enumerate(hi):
        yield row[1]['text']
count = 0
for row in hi_text():
    print(row)
    count += 1
    if count == 3: break

ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है
अवधि को हाइलाइट रकें


In [ ]:
hitext = hi_text()
hitext

<generator object hi_text at 0x7f5bbbec5eb0>

In [ ]:
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.8 MB/s eta 0:00:00


In [ ]:
#wordpiece tokenizer
from tokenizers import models,Tokenizer
tokenizer = Tokenizer(models.WordPiece(unk_token='[UNK]'))

In [ ]:
#normalizer
from tokenizers import normalizers
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.Lowercase(), normalizers.NFKD()]
)

In [ ]:
#pretokenization to seperate words
from tokenizers import pre_tokenizers
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

In [ ]:
#training using trainer
from tokenizers import trainers

trainer = trainers.WordPieceTrainer(
    vocab_size=30_000,
    special_tokens=['[UNK]', '[PAD]', '[CLS]', '[SEP]', '[MASK]'],
    min_frequency=2,
    continuing_subword_prefix='##'
)

In [ ]:
tokenizer.train_from_iterator(hitext, trainer=trainer)

In [ ]:
#post processing 
from tokenizers import processors

# first we get the token ID values (defined in the vocab) for CLS and SEP
cls_id = tokenizer.token_to_id('[CLS]')
sep_id = tokenizer.token_to_id('[SEP]')

# then setup the post processing step with TemplateProcessing
tokenizer.post_processor = processors.TemplateProcessing(
    single=f'[CLS]:0 $A:0 [SEP]:0',
    pair=f'[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1',
    special_tokens=[
        ('[CLS]', cls_id),
        ('[SEP]', sep_id)
    ]
)

In [ ]:
#decoder
from tokenizers import decoders

tokenizer.decoder = decoders.WordPiece(prefix='##')

In [ ]:
!pip install transformers

In [ ]:
from transformers import PreTrainedTokenizerFast

# load the tokenizer in a transformers tokenizer instance
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token='[UNK]',
    pad_token='[PAD]',
    cls_token='[CLS]',
    sep_token='[SEP]',
    mask_token='[MASK]'
)

# save the tokenizer
tokenizer.save_pretrained('bert-base-hi')

('bert-base-hi/tokenizer_config.json',
 'bert-base-hi/special_tokens_map.json',
 'bert-base-hi/tokenizer.json')

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('bert-base-hi')

In [ ]:
tokenizer('अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें')

{'input_ids': [2, 814, 3428, 710, 1698, 12619, 11060, 714, 1615, 2074, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
#download the tokenizer
# !zip -r /content/bert-base-hi.zip /content/bert-base-hi
# from google.colab import files
# files.download('/content/bert-base-hi.zip') 

  adding: content/bert-base-hi/ (stored 0%)
  adding: content/bert-base-hi/tokenizer_config.json (deflated 38%)
  adding: content/bert-base-hi/tokenizer.json (deflated 76%)
  adding: content/bert-base-hi/special_tokens_map.json (deflated 42%)
